In [ ]:
from keras import applications
from evec_scan.tools import evectools as evt
import pandas as pd

InstanceGen = evt.InstanceGenerator(instance_dir='evec_scan/instances/test1', vid_file='vid3.mp4', instance_name='test_instance3')
print('Instance created\nStructure: ', InstanceGen.get_structure()) 
InstanceGen.generate_instance_frames(frame_quant=3) #works
print('Instance frames generated\nStructure: ', InstanceGen.get_structure())
InstanceGen.get_descriptions(ollama_model_list=['moondream', 'llava-llama3'])
print('Instance frames described\nStructure: ', InstanceGen.get_structure())
InstanceGen.get_description_embeddings(ollama_model_list=['mxbai-embed-large','nomic-embed-text', 'all-minilm'])
print('Instance descriptions embedded\nStructure: ', InstanceGen.get_structure())
structure = InstanceGen.get_structure()
structure['image_embedding_models'] = []
structure['image_embedding_vectors'] = []
structure['image_embedding_vectors_shape'] = []

for frame_path in structure['frame_paths']:
    frame_evec = evt.ImgEVecScanner(frame_path=frame_path)
    # frame_evec.add_model(Keras_applications_model=applications.VGG19)
    # frame_evec.change_preprocess_map(...)
    # frame_evec.change_models_to_use(...)
    # frame_evec.get_model_list(...)
    # frame_evec.get_preprocess_map(...)
    ImgEvecDict = frame_evec.get_models_evecs(lin_method='GAP')
    print('image embedding data', ImgEvecDict)
    structure['image_embedding_models'].append(ImgEvecDict['model_name'])
    structure['image_embedding_vectors'].append(ImgEvecDict['embedding_vector'])
    structure['image_embedding_vectors_shape'].append(ImgEvecDict['vector_shape'])


Instance frames described
Structure:  {'frame_number': [7827, 20155, 30097], 'frame': [Array([[[ 28,  28,  28],
        [ 28,  28,  28],
        [ 26,  26,  26],
        ...,
        [112,  91,  50],
        [112,  91,  50],
        [112,  91,  50]],

       [[ 28,  28,  28],
        [ 27,  27,  27],
        [ 26,  26,  26],
        ...,
        [112,  91,  50],
        [112,  91,  50],
        [112,  91,  50]],

       [[ 28,  28,  28],
        [ 26,  26,  26],
        [ 27,  27,  27],
        ...,
        [112,  91,  50],
        [112,  91,  50],
        [112,  91,  50]],

       ...,

       [[189, 177, 165],
        [189, 177, 165],
        [188, 177, 163],
        ...,
        [120,  97,  69],
        [119,  96,  66],
        [118,  95,  65]],

       [[188, 176, 164],
        [188, 176, 164],
        [188, 177, 163],
        ...,
        [120,  97,  69],
        [120,  97,  67],
        [120,  97,  67]],

       [[189, 177, 165],
        [188, 176, 164],
        [189, 178, 164],


ResponseError: model "mxbai-embed-large" not found, try pulling it first

Instance descriptions embedded
Structure:  {'frame_number': [7827, 20155, 30097], 'frame': [Array([[[ 28,  28,  28],
        [ 28,  28,  28],
        [ 26,  26,  26],
        ...,
        [112,  91,  50],
        [112,  91,  50],
        [112,  91,  50]],

       [[ 28,  28,  28],
        [ 27,  27,  27],
        [ 26,  26,  26],
        ...,
        [112,  91,  50],
        [112,  91,  50],
        [112,  91,  50]],

       [[ 28,  28,  28],
        [ 26,  26,  26],
        [ 27,  27,  27],
        ...,
        [112,  91,  50],
        [112,  91,  50],
        [112,  91,  50]],

       ...,

       [[189, 177, 165],
        [189, 177, 165],
        [188, 177, 163],
        ...,
        [120,  97,  69],
        [119,  96,  66],
        [118,  95,  65]],

       [[188, 176, 164],
        [188, 176, 164],
        [188, 177, 163],
        ...,
        [120,  97,  69],
        [120,  97,  67],
        [120,  97,  67]],

       [[189, 177, 165],
        [188, 176, 164],
        [189, 178, 1

ValueError: All arrays must be of the same length

In [5]:
structure

{'frame_number': [7827, 20155, 30097],
 'frame': [Array([[[ 28,  28,  28],
          [ 28,  28,  28],
          [ 26,  26,  26],
          ...,
          [112,  91,  50],
          [112,  91,  50],
          [112,  91,  50]],
  
         [[ 28,  28,  28],
          [ 27,  27,  27],
          [ 26,  26,  26],
          ...,
          [112,  91,  50],
          [112,  91,  50],
          [112,  91,  50]],
  
         [[ 28,  28,  28],
          [ 26,  26,  26],
          [ 27,  27,  27],
          ...,
          [112,  91,  50],
          [112,  91,  50],
          [112,  91,  50]],
  
         ...,
  
         [[189, 177, 165],
          [189, 177, 165],
          [188, 177, 163],
          ...,
          [120,  97,  69],
          [119,  96,  66],
          [118,  95,  65]],
  
         [[188, 176, 164],
          [188, 176, 164],
          [188, 177, 163],
          ...,
          [120,  97,  69],
          [120,  97,  67],
          [120,  97,  67]],
  
         [[189, 177, 165],
   